In [ ]:
! pip install git+https://gitlab.aicrowd.com/yoogottamk/aicrowd-cli.git >/dev/null
%load_ext aicrowd.magic

  Running command git clone -q https://gitlab.aicrowd.com/yoogottamk/aicrowd-cli.git /tmp/pip-req-build-1dexf4yq


In [ ]:
API_KEY = "47db8507b8e1dda36d071b1b32a27354" 
%aicrowd login --api-key $API_KEY

API Key valid
Saved API Key successfully!


In [ ]:
%aicrowd dataset download -c tile-slider -j 3
%aicrowd dataset download --challenge tile-slider -j 3

In [ ]:
! unzip -q dataset.zip

In [ ]:
import os
import random
import tqdm
import pandas as pd
import numpy as np

In [ ]:
DATA_PATH = "dataset/" #path where data is stored

In [ ]:
file = open(os.path.join(DATA_PATH,os.listdir(DATA_PATH)[0]), "r")
print(file.read())

40 20
....#..#.#....#..#.#
......#...##....#.##
....#..#....##....##
.....##.##.#####.#..
.....#........###..#
.#......#.##.#.#..#.
.....#.#...#.#..#...
......#........##...
...##....#...##..##.
.#.#.#.#.##..#....##
.#.#..#...#...###...
..#....#.###.#....#.
....#####.####.....#
#..............#..##
.#..#........#.##...
...##....##.#.#.#..#
##.....#....#..#...#
.....#...........##.
.#.##.#.#.###...#...
##..##...#....#.....
#........#.###.#....
#......#.#..##.#....
#....#....##.......#
..#.......##...##...
.#.......#.........#
.##...#...#..#####.#
.#.....#...#....#.##
..#.#...##.###.##..#
.#.#.....#..###.....
.#..#.##.#....#.....
......###.......#.##
................##..
#.......#.#.......##
..#..#.........##...
##..####...###.#.##.
##.......#...#...#..
.#...#.#...#......#.
#.###.#.#.........##
..#.#.#.#...##.#.#.#
..##..#.......#....#
3
38 10 38 14
36 4 31 10
8 5 14 3



In [ ]:
class GridState:
    def __init__(self, input_file):
        self.UP = (-1, 0)
        self.DOWN = (1, 0)
        self.LEFT = (0, -1)
        self.RIGHT = (0, 1)
        self.load(input_file)

    def load(self, input_file):
        file = open(input_file, 'r')
        # Get the Grid
        self.n, self.m = list(map(int, file.readline().strip().split()))
        self.grid = np.array(
            [[cell == "." for cell in file.readline().strip()] for _ in range(self.n)])
        k = int(file.readline().strip())
        self.tiles, self.targets = [], []
        for _ in range(k):
            line = list(map(int, file.readline().strip().split()))
            self.tiles.append((line[0], line[1]))
            self.targets.append((line[2], line[3]))
        file.close()

    def move_value(self, direction):
        if direction == 'U':
            return self.UP
        elif direction == 'D':
            return self.DOWN
        elif direction == 'L':
            return self.LEFT
        elif direction == 'R':
            return self.RIGHT

    def move(self, move):
        delta_r, delta_c = self.move_value(move)
        flag = 0
        for _ in range(max(self.n, self.m) + 1):
            for idx, tile in enumerate(self.tiles):
                next_r, next_c = tile[0] + delta_r, tile[1] + delta_c
                if 0 <= next_r < self.n and 0 <= next_c < self.m and \
                        self.grid[next_r, next_c] and (next_r, next_c) not in self.tiles:
                    flag = 1
                    tile = tile[0] + delta_r, tile[1] + delta_c
                self.tiles[idx] = tile
            # no change in any tile
            if flag == 0:
                break

In [ ]:
def create_random_moves(moves_length):
    seq = ['U', 'D', 'L', 'R']
    moves = ''.join(random.choices(seq, k=moves_length))
    return moves

In [ ]:
solved = 0
moves_length = 1000
lst = sorted(os.listdir(DATA_PATH))
total = len(lst)
df = pd.DataFrame(columns = {'filename', 'moves'})

for file in tqdm.tqdm(lst):
  path = os.path.join(DATA_PATH, file)
  obj = GridState(path)
  moves = create_random_moves(moves_length)
  for direction in moves:
      obj.move(direction)
      if obj.tiles == obj.targets:
          # print(obj.tiles, obj.targets)
          solved += 1
          # print("Puzzle Solved")
          break
  df = df.append({'filename':file, 'moves': moves}, ignore_index = True)

100%|██████████| 2000/2000 [01:28<00:00, 22.72it/s]


In [ ]:
df.head()

,filename,moves
0,0001.txt,DDURDDRRUURDLDLLUDULDLUDRUULDRUDLRDRDRRDDUDDDL...
1,0002.txt,DRDDDLULLLLRUURRLRUUDRLLUDULLDDURRDLURLRURUDRR...
2,0003.txt,LDRURDLUDURDLDRRUURDRDLLLDURUURDUDDLRLLRUUUURU...
3,0004.txt,RUUUULDDRDRRLDRRDRRRULUDUUUDRUDRLDUDLULLUUULDR...
4,0005.txt,LRDLRUURDURLDUDUULLRULRURDLLLDLLDRLDDULDRDRLDD...


In [ ]:
print("Number of puzzles solved is :" ,solved)
print("Percentage of puzzles solved :" , (solved/total)*100)

Number of puzzles solved is : 411
Percentage of puzzles solved : 20.549999999999997


In [ ]:
submission = pd.DataFrame(df)
submission.to_csv('submission.csv',index=False)